In [2]:
import json

json_path = "../../../synced_data/ShareGPT_V3_unfiltered_cleaned_split_no_imsorry.json"
# json_path = "./test.json"

with open(json_path, "r") as f:
    sharegpt = json.load(f)
# print(sharegpt)
print("Qids: ", len(sharegpt))

Qids:  94145


In [3]:
### merge conversations by its user id
merge_conversations = {}
for item in sharegpt:
    user_id = item["id"].split("_")[0]
    if user_id not in merge_conversations:
        merge_conversations[user_id] = item["conversations"]
    else:
        merge_conversations[user_id] += item["conversations"]

# concated_conversations = []
# for k, v in merge_conversations.items():
#     concated_conversations[k] = 

print(len(merge_conversations))
print("conver: ", merge_conversations["QWJhYvA"][0])
print("conver: ", merge_conversations["QWJhYvA"][1])
print("conver: ", merge_conversations["QWJhYvA"][2])


50142
conver:  {'from': 'human', 'value': "Summarize the main ideas of Jeff Walker's Product Launch Formula into bullet points as it pertains to a growth marketing agency implementing these strategies and tactics for their clients..."}
conver:  {'from': 'gpt', 'value': "Here are the main ideas of Jeff Walker's Product Launch Formula that can be applied by a growth marketing agency for their clients:\n\n1. Identify the target audience and their needs: Understand the ideal customer for the product or service, and create a messaging that resonates with them.\n2. Pre-launch: Build anticipation and excitement for the launch by creating buzz, gathering testimonials and case studies, and using social media to create awareness.\n3. Launch: Use a well-crafted launch sequence to maximize sales and conversions. This can include offering bonuses, creating scarcity, and using a deadline to create urgency.\n4. Post-launch: Follow up with customers, gather feedback, and continue to provide value to k

In [4]:
question = {}
answer = {}
all_q = []
all_a = []
cnt = 0
for user_id, conversations in merge_conversations.items():
    # print(user_id, len(conversations))
    prev_from = ''
    q, a = [], []

    for conver in conversations:
        c = conver["value"]

        if cnt % 2 == 0 and conver["from"] != 'human':  # first conversation should be from human
            # print("Error    : ", id , cnt)
            continue
        if c == "":  # empty string
            continue
            
        c = c.replace("\n", " ")

        if conver["from"] == 'human':
            if prev_from == 'human':
                q[-1] += " " + c
            else:
                q.append(c)
                cnt += 1
            prev_from = 'human'
        elif conver["from"] == 'gpt' or conver["from"] == 'bing' or \
            conver["from"] == 'chatgpt' or conver["from"] == "bard":
            # print("id", item["id"], prev_from)
            # print(len(a))
            if prev_from == 'human':
                a.append(c)
                cnt += 1
            # else:
                # a[-1] += " " + c
            prev_from = ''
        else:
            raise Exception("Unknown conversation from: ", conver["from"])

    min_len = min(len(q), len(a))
    q = q[:min_len]
    a = a[:min_len]

    question[user_id] = q
    answer[user_id] = a
    all_q += q
    all_a += a

print("Dict Questions:", len(question))
print("Dict Answers:", len(answer))
print("Questions: ", len(all_q))
print("Answers: ", len(all_a))


Dict Questions: 50142
Dict Answers: 50142
Questions:  330244
Answers:  330244


In [5]:
# from sacrerouge.metrics import Rouge

# summary = 'Dan walked to the bakery this morning.'
# reference = 'Dan went to buy scones earlier this morning.'

# rouge = Rouge(max_ngram=2)
# rouge.score(summary, [reference])